In [3]:
import pandas as pd

# Ruta corregida (subimos un nivel desde notebooks/)
raw_path = "../data/raw/SalesFINAL12312016.csv"

# Cargar dataset
sales_raw = pd.read_csv(raw_path)

# Ver info básica
print("Shape (filas, columnas):", sales_raw.shape)
print("\nPrimeras columnas:", list(sales_raw.columns)[:10])

# 1) Nulos por columna (top 10)
print("\nNulos por columna (top 10):")
print(sales_raw.isna().sum().sort_values(ascending=False).head(10))

# 2) Duplicados exactos
print("\nFilas duplicadas exactas:", sales_raw.duplicated().sum())

# 3) Exploración rápida de columnas típicas con problemas
for col in ["Brand", "Classification", "Size"]:
    if col in sales_raw.columns:
        print(f"\nColumna: {col}")
        print(f"Valores únicos: {sales_raw[col].nunique()}")
        print(sales_raw[col].astype(str).value_counts().head(5))
    else:
        print(f"⚠️ La columna {col} no existe en este CSV")

Shape (filas, columnas): (1048575, 14)

Primeras columnas: ['InventoryId', 'Store', 'Brand', 'Description', 'Size', 'SalesQuantity', 'SalesDollars', 'SalesPrice', 'SalesDate', 'Volume']

Nulos por columna (top 10):
InventoryId      0
Store            0
Brand            0
Description      0
Size             0
SalesQuantity    0
SalesDollars     0
SalesPrice       0
SalesDate        0
Volume           0
dtype: int64

Filas duplicadas exactas: 0

Columna: Brand
Valores únicos: 7658
Brand
3606    2077
8111    2064
1892    2016
4261    1969
5111    1962
Name: count, dtype: int64

Columna: Classification
Valores únicos: 2
Classification
1    611791
2    436784
Name: count, dtype: int64

Columna: Size
Valores únicos: 40
Size
750mL    593098
1.75L    172222
50mL     101816
1.5L      67189
375mL     52659
Name: count, dtype: int64


In [4]:
import re
import pandas as pd
from pathlib import Path

def normalize_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = (
        df.columns.str.strip()
                  .str.lower()
                  .str.replace(r"\s+", "_", regex=True)
                  .str.replace("[^a-z0-9_]", "", regex=True)
    )
    return df

def normalize_brand(s: pd.Series) -> pd.Series:
    # quita espacios dobles, trim, upper
    return (s.astype(str)
              .str.strip()
              .str.replace(r"\s+", " ", regex=True)
              .str.upper())

def normalize_classification(s: pd.Series) -> pd.Series:
    return (s.astype(str)
              .str.strip()
              .str.replace(r"\s+", " ", regex=True)
              .str.upper())

def size_to_ml(val) -> float | None:
    """Convierte '750mL', '1 L', '12oz' -> mililitros (float). Si no puede, None."""
    if pd.isna(val):
        return None
    v = str(val).strip().lower().replace(" ", "")
    # n.nnL o nL
    m = re.fullmatch(r"(\d+(\.\d+)?)l", v)
    if m:
        return float(m.group(1)) * 1000
    # nnnml
    m = re.fullmatch(r"(\d+(\.\d+)?)ml", v)
    if m:
        return float(m.group(1))
    # nnoz
    m = re.fullmatch(r"(\d+(\.\d+)?)oz", v)
    if m:
        return float(m.group(1)) * 29.57
    # nnn (solo número) -> deja como None (no sabemos unidad)
    return None

In [5]:
# 1) cargar
sales = pd.read_csv("../data/raw/SalesFINAL12312016.csv")

# 2) normalizar nombres de columnas
sales = normalize_columns(sales)

# 3) columnas texto clave: brand / classification / description (si existen)
for col in ["brand", "classification", "description"]:
    if col in sales.columns:
        if col == "brand":
            sales[col] = normalize_brand(sales[col])
        elif col == "classification":
            sales[col] = normalize_classification(sales[col])
        else:
            sales[col] = sales[col].astype(str).str.strip()

# 4) convertir size -> size_ml (nueva columna)
if "size" in sales.columns:
    sales["size_ml"] = sales["size"].apply(size_to_ml)

# 5) tipos básicos
for c in ["salesquantity", "salesdollars", "salesprice", "volume"]:
    if c in sales.columns:
        # intento de conversión segura a numérico
        sales[c] = pd.to_numeric(sales[c], errors="coerce")

if "salesdate" in sales.columns:
    sales["salesdate"] = pd.to_datetime(sales["salesdate"], errors="coerce")

# 6) duplicados exactos fuera
sales = sales.drop_duplicates()

# 7) guardar en processed
out = Path("../data/processed")
out.mkdir(parents=True, exist_ok=True)
sales_out_csv = out / "sales_clean.csv"
sales.to_csv(sales_out_csv, index=False)

# parquet (si falla, lo ignoramos)
try:
    sales.to_parquet(out / "sales_clean.parquet", index=False)
except Exception as e:
    print("Parquet no disponible (ok):", e)

print("✅ Limpieza básica de SalesFINAL completa.")
print("Archivo:", sales_out_csv)
print("Shape limpio:", sales.shape)
sales.head(3)

✅ Limpieza básica de SalesFINAL completa.
Archivo: ../data/processed/sales_clean.csv
Shape limpio: (1048575, 15)


,inventoryid,store,brand,description,size,salesquantity,salesdollars,salesprice,salesdate,volume,classification,excisetax,vendorno,vendorname,size_ml
0,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,16.49,16.49,2016-01-01,750,1,0.79,12546,JIM BEAM BRANDS COMPANY,750.0
1,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,2,32.98,16.49,2016-01-02,750,1,1.57,12546,JIM BEAM BRANDS COMPANY,750.0
2,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,16.49,16.49,2016-01-03,750,1,0.79,12546,JIM BEAM BRANDS COMPANY,750.0


In [6]:
clean = pd.read_csv("../data/processed/sales_clean.csv")
print("Shape:", clean.shape)
print("\nNulos por columna (top 10):")
print(clean.isna().sum().sort_values(ascending=False).head(10))

for col in ["brand", "classification", "size", "size_ml"]:
    if col in clean.columns:
        print(f"\n{col}: únicos ->", clean[col].nunique())

Shape: (1048575, 15)

Nulos por columna (top 10):
size_ml          27336
inventoryid          0
store                0
brand                0
description          0
size                 0
salesquantity        0
salesdollars         0
salesprice           0
salesdate            0
dtype: int64

brand: únicos -> 7658

classification: únicos -> 2

size: únicos -> 40

size_ml: únicos -> 21


In [7]:
import pandas as pd

sales = pd.read_csv("../data/processed/sales_clean.csv")
purch = pd.read_csv("../data/processed/purchases_clean.csv")
inv_b = pd.read_csv("../data/processed/inventory_beg_clean.csv")
inv_e = pd.read_csv("../data/processed/inventory_end_clean.csv")
invp = pd.read_csv("../data/processed/invoice_purchases_clean.csv")
prices = pd.read_csv("../data/processed/prices_clean.csv")

print("sales:", sales.shape, "purchases:", purch.shape, "invoice:", invp.shape)
print("inv_b:", inv_b.shape, "inv_e:", inv_e.shape, "prices:", prices.shape)

# claves típicas presentes
for df, name in [(sales,"sales"),(purch,"purchases"),(invp,"invoice_purchases")]:
    print(f"\n[{name}] columnas disponibles:", [c for c in df.columns if c in ("inventoryid","store","brand","classification","size_ml","salesdate","quantity","price","cost")])

# ejemplo de consistencia: ¿existen inventoryid-store de ventas en inventarios?
if all(c in sales.columns for c in ["inventoryid","store"]) and all(c in inv_b.columns for c in ["inventoryid","store"]):
    keys_sales = sales[["inventoryid","store"]].drop_duplicates()
    keys_invb = inv_b[["inventoryid","store"]].drop_duplicates()
    miss = len(keys_sales.merge(keys_invb, on=["inventoryid","store"], how="left", indicator=True).query("_merge=='left_only'"))
    print("\nPairs (inventoryid, store) de sales NO presentes en inv_beg:", miss)

# nulos clave
for df, name in [(sales,"sales"),(purch,"purchases"),(invp,"invoice_purchases"),(inv_b,"inv_beg"),(inv_e,"inv_end"),(prices,"prices")]:
    print(f"\n[{name}] nulos top:")
    print(df.isna().sum().sort_values(ascending=False).head(8))

sales: (1048575, 15) purchases: (2372474, 17) invoice: (5543, 10)
inv_b: (206529, 9) inv_e: (224489, 9) prices: (12261, 9)

[sales] columnas disponibles: ['inventoryid', 'store', 'brand', 'quantity', 'classification', 'size_ml']

[purchases] columnas disponibles: ['inventoryid', 'store', 'brand', 'quantity', 'classification', 'size_ml']

[invoice_purchases] columnas disponibles: ['quantity']

Pairs (inventoryid, store) de sales NO presentes en inv_beg: 2872

[sales] nulos top:
size_ml        27336
inventoryid        0
store              0
brand              0
productname        0
size               0
quantity           0
totalamount        0
dtype: int64

[purchases] nulos top:
size_ml           68623
size                  3
receivingdate         0
classification        0
totalamount           0
quantity              0
unitprice             0
paydate               0
dtype: int64

[invoice_purchases] nulos top:
approval       5169
vendorid          0
vendorname        0
invoicedate     

# 01 — Limpieza y normalización

Este notebook acompaña el pipeline `src/transform_template.py`.  
Propósito:
- Documentar **qué limpiezas** se aplican (nombres de columnas, textos, tamaños, fechas).
- Mostrar **ejemplos** de antes/después.
- Confirmar **archivos de salida** en `data/processed/` y diccionarios.

## Reglas de limpieza aplicadas

1. **Columnas** → `snake_case`, sin espacios, sin símbolos.
2. **Textos**  
   - `brand`: normalizado (trim, espacios, mayúsculas según regla)  
   - `classification`, `description`, `vendorname`: limpiar espacios dobles/ruido
3. **Tallas (`size`)** → convertir a mililitros en `size_ml` (L → mL, oz → mL)
4. **Fechas** → `*_date` parseadas a tipo fecha
5. **Duplicados** → filas exactas eliminadas
6. **Diccionario de datos** → `*_dictionary.csv` con `dtype` y `nulls`

## Ejemplos antes → después

- **brand**: `"  Diageo  "` → `"DIAGEO"`
- **size**: `"1.75L"` → `1750` (en `size_ml`)
- **classification**: `"SPIRITS  /  WHISKY "` → `"SPIRITS / WHISKY"`
- **fechas**: `"2016-01-02"` → `datetime64[ns]`

## Outputs esperados

- `data/processed/sales_clean.csv`
- `data/processed/purchases_clean.csv`
- `data/processed/invoice_purchases_clean.csv`
- `data/processed/inventory_beg_clean.csv`
- `data/processed/inventory_end_clean.csv`
- `data/processed/prices_clean.csv`
- Diccionarios: `data/processed/*_dictionary.csv`

## Notas

- Este notebook es **explicativo**; el flujo oficial está en `src/transform_template.py`
- Si necesitas actualizar reglas, modifícalas en el script y vuelve a correr:
  ```bash
  python -m src.transform_template